In [14]:
# lets first get our data imports and actual data we will work with
# NO PYTORCH 
from datasets import load_dataset

from typing import Any, Dict, List, Literal, Optional, Sequence, Tuple, Union
import tinygrad

In [3]:
# let's get teh dataset first, we will not use dataloaders so we may need to build one from scratch? this should be fun who knows?
name = "fashion_mnist"
dsd = load_dataset(name)
dsd

Found cached dataset fashion_mnist (/Users/diegomedina-bernal/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48)
100%|██████████| 2/2 [00:00<00:00, 385.03it/s]


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [4]:
# just split for now, we know this, and x, y so we dont keep writing the names
train, test = dsd["train"], dsd["test"]
x, y = 'image', 'label'

In [12]:
# imports for dataloader to work
from PIL import Image
try: import accimage
except ImportError: accimage = None
import numpy as np

In [15]:
def is_pil_image(img: Any) -> bool:
    if accimage is not None: return isinstance(img, (Image.Image, accimage.Image))
    else: return isinstance(img, Image.Image)
def is_numpy(img: Any) -> bool: return isinstance(img, np.ndarray)

In [11]:
def to_tensor(pic) -> Tensor:
    """Convert a ``PIL Image`` or ``numpy.ndarray`` to Tinygrad tensor.
    thats right a tinygrad tensor not those damn pytorch ones
    """

    if not (is_pil_image(pic) or _is_numpy(pic)):
        raise TypeError(f"pic should be PIL Image or ndarray. Got {type(pic)}")

    if _is_numpy(pic) and not _is_numpy_image(pic):
        raise ValueError(f"pic should be 2/3 dimensional. Got {pic.ndim} dimensions.")

    default_float_dtype = torch.get_default_dtype()

    if isinstance(pic, np.ndarray):
        # handle numpy array
        if pic.ndim == 2:
            pic = pic[:, :, None]

        img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()
        # backward compatibility
        if isinstance(img, torch.ByteTensor):
            return img.to(dtype=default_float_dtype).div(255)
        else:
            return img

    if accimage is not None and isinstance(pic, accimage.Image):
        nppic = np.zeros([pic.channels, pic.height, pic.width], dtype=np.float32)
        pic.copyto(nppic)
        return torch.from_numpy(nppic).to(dtype=default_float_dtype)

    # handle PIL Image
    mode_to_nptype = {"I": np.int32, "I;16": np.int16, "F": np.float32}
    img = torch.from_numpy(np.array(pic, mode_to_nptype.get(pic.mode, np.uint8), copy=True))

    if pic.mode == "1":
        img = 255 * img
    img = img.view(pic.size[1], pic.size[0], F_pil.get_image_num_channels(pic))
    # put it from HWC to CHW format
    img = img.permute((2, 0, 1)).contiguous()
    if isinstance(img, torch.ByteTensor):
        return img.to(dtype=default_float_dtype).div(255)
    else:
        return img

In [10]:
## Okay we will slightly cheat, it seems that tinygrad doesnt have dataloaders or datasets so lets use pytorch ones for now and maybe we build our own? maybe? 
import torchvision.transforms.functional as TF

ModuleNotFoundError: No module named 'torchvision'